# Graded Challenge 3

**Name: Prabaningrum Hendraswari Buditjahjono**

**Batch: 001-HCK**

## Working Area

In [18]:
import pandas as pd
from scipy import stats

In [19]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.cloud import bigquery

project_id = "elevated-surge-360509"
client = bigquery.Client(project=project_id)

Authenticated


1. What percentage of the number of users are retained from January - June 2022 of the total existing users? (Assuming the definition of retention is a minimum of 6 transactions during the period) What conclusions/insights can you give?

In [20]:
q = client.query('''
SELECT count(case when loyalty = 'ya' then 1 else null end)/count(user_id)*100 as percentage FROM
  (SELECT user_id, count(order_id),
  CASE WHEN count(order_id) >= 3 THEN 'ya'
  ELSE 'tidak' end as loyalty FROM
    (SELECT * FROM 
      (SELECT * FROM `bigquery-public-data.thelook_ecommerce.orders`
      WHERE created_at >= '2022-01-01' AND created_at <= '2022-06-30')
    WHERE status = 'Complete')
  GROUP BY user_id)
''').to_dataframe()
q

,percentage
0,0.194375


In this case, we define user retention as users who use The Look's service continually and has completed transactions without returning/cancelling. Thus, we are only calculating user retention based on the customers who has had completed transactions.

Based on the calculation, it can be inferred that the retention is very low at 0.41. However, we also have to keep in mind that since we're only calculating the retention based on the customers who has multiple (>= 3) completed transactions, we are calculating the data that has gone through churning process. 

While it would depend on the focus of the company whether they would like to maintain their business by prioritizing their user retention or other things, the user retention rate could be improved by analyzing what factors might affect the churning rate and the user experience.

2. Provide information on the average, minimum and maximum number of transactions made by loyal users. (View only in one dataframe and based on one SQL select query) What information can you provide?

In [21]:
q1 = client.query('''
SELECT avg(total) AS mean,
min(total) as minimum,
max(total) as maximum
FROM
  (SELECT orders AS total FROM
    (SELECT user_id, count(order_id) as orders,
    CASE WHEN count(order_id) >= 3 THEN 'ya'
    ELSE 'tidak' end as loyalty FROM
      (SELECT * FROM 
        (SELECT * FROM `bigquery-public-data.thelook_ecommerce.orders`
        WHERE created_at >= '2022-01-01' AND created_at <= '2022-06-30')
      WHERE status = 'Complete')
    GROUP BY user_id HAVING loyalty = 'ya'))
''').to_dataframe()
q1

,mean,minimum,maximum
0,3.0,3,3


Based on the finding, it can be found that the number of average transaction and the minimum transaction are awfully similar since the minimum transactions of a loyal customer is 3, the maximum is 4, and the average is 3.05. This means that even the loyal customers on The Look doesn't do a lot of transactions.

3. Provide information on average, minimum, and maximum money spent by loyal users. (Display only in one dataframe and based on one SQL select query) What information can you provide?

In [22]:
q2 = client.query('''
SELECT avg(spent) AS mean,
min(spent) AS minimum,
max(spent) AS maximum
  FROM
  (SELECT sum(sale_price) as spent, user_id FROM
    (SELECT oi.order_id, sale_price, o.user_id 
    FROM `bigquery-public-data.thelook_ecommerce.order_items` as oi
    JOIN
      (SELECT user_id, orders FROM
          (SELECT user_id, count(order_id) as orders,
          CASE WHEN count(order_id) >= 3 THEN 'ya'
          ELSE 'tidak' end as loyalty FROM
            (SELECT * FROM 
              (SELECT * FROM `bigquery-public-data.thelook_ecommerce.orders`
              WHERE created_at >= '2022-01-01' AND created_at <= '2022-06-30')
            WHERE status = 'Complete')
        GROUP BY user_id HAVING loyalty = 'ya'))
    as o ON oi.user_id = o.user_id)
  GROUP BY user_id)
''').to_dataframe()
q2

,mean,minimum,maximum
0,383.631766,140.740002,1054.960005


The result once again shows that the average spending is far closer to the minimum spending with the minimum spending being 118.66, the maximum being 1071.2, and the average being 338.64. This confirms the finding before that the loyal customers of The Look isn't doing that much transactions and isn't spending a lot.

With the comparison between the minimum, maximum, and mean numbers of transations being done and the amount of money spent by the loyal customers, it can be assumed that perhaps the price of thhe items sold by The Look is varied. However, it needs further exploration such as finding the median and standard deviation to verify it. 

4. From which traffic sources do most loyal users access The Look platform? What information can you provide?

In [23]:
q3 = client.query('''
(SELECT ec.traffic_source, count(o.user_id) 
    FROM `bigquery-public-data.thelook_ecommerce.events` as ec
    JOIN
      (SELECT user_id, orders FROM
          (SELECT user_id, count(order_id) as orders,
          CASE WHEN count(order_id) >= 3 THEN 'ya'
          ELSE 'tidak' end as loyalty FROM
            (SELECT * FROM 
              (SELECT * FROM `bigquery-public-data.thelook_ecommerce.orders`
              WHERE created_at >= '2022-01-01' AND created_at <= '2022-06-30')
            WHERE status = 'Complete')
          GROUP BY user_id HAVING loyalty = 'ya'))
as o ON ec.user_id = o.user_id
GROUP BY ec.traffic_source)
''').to_dataframe()
q3

,traffic_source,f0_
0,Email,264
1,Adwords,236
2,Organic,57
3,Facebook,79
4,YouTube,69


The table shows that most of the traffic source of The Look's loyal customers mostly comes from email, and followed by adwords. Due to this, if The Look wants to improve their sales based on their loyal customers, they could do it through maintaining relations with their loyal customers using email and keeps the allocation for ads constant (always available).

5. Is there a relationship between traffic sources and whether a user is loyal or not? (Use the chi-square test to test the relationship) Hint: The output query table only has two columns, namely 'traffic source' and 'loyal' (column 'loyal' contains only yes and no

In [24]:
q4 = client.query('''
(SELECT ec.traffic_source, o.loyalty as loyal 
    FROM `bigquery-public-data.thelook_ecommerce.events` as ec
    JOIN
       (SELECT user_id, orders, loyalty FROM
          (SELECT user_id, count(order_id) as orders,
          CASE WHEN count(order_id) >= 3 THEN 'ya'
          ELSE 'tidak' end as loyalty FROM
            (SELECT * FROM 
              (SELECT * FROM `bigquery-public-data.thelook_ecommerce.orders`
              WHERE created_at >= '2022-01-01' AND created_at <= '2022-06-30')
            WHERE status = 'Complete')
          GROUP BY user_id))
as o ON ec.user_id = o.user_id)
''').to_dataframe()
q4

,traffic_source,loyal
0,Email,tidak
1,Email,tidak
2,Email,tidak
3,Email,tidak
4,Email,tidak
...,...,...
181161,Facebook,tidak
181162,Facebook,tidak
181163,Facebook,tidak
181164,Facebook,tidak


In [25]:
contingency=pd.crosstab(q4["traffic_source"],q4["loyal"])
contingency

loyal,tidak,ya
traffic_source,,
Adwords,53911,236
Email,80391,264
Facebook,19195,79
Organic,8930,57
YouTube,18034,69


**H0= traffic source has no significant correlation to users' loyalty**

**H1= traffic source has a significant correlation to users' loyalty**

In [28]:
stat, p, dof, expected = stats.chi2_contingency(contingency)
alpha = 0.05
print("p value is " + str(p))
if p <= alpha:
    print('Dependent')
else:
    print('Independent')

p value is 4.645422816122699e-05
Dependent


Since the p value of the test is below 0.05, it means that the vairables are dependent or that statistically, there is a significant correlation between the traffic source of The Look, and the loyalty of the customers, thus rejecting the null hypothesis.

# Problem Statement

In this analysis, we are trying to evaluate the transaction activity of an online platform called The Look. The fact that The Look has low user retention could indicate that they're not doing good considering that they're one of the largest fashion e-commerce on Mars. With this, their goal should be to increase their user retention 2% by the end of the year.

## Conclusions

From the analysis, it's known that the user retention from January to June 2022 is very low at 0.19% compared to the whole data. This could be because there might not be a lot of transactions that has been completed whether the customer ended up receiving it or returning it. It's also found that even the loyal customers of The Look is not doing a lot of transactions nor are they spending huge amounts of money. The last analysis also shows that traffic source of how people access The Look might affect their loyalty.

If The Look is trying to increase their profit through customer retention, they might have to analyze the behavior of the customers further, both loyal or not and compare them in order to discover the patterns of how they might be able to attract loyal customers. On the other hand, since the customer retention is low, they also might want to focus on other methods of keeping their business afloat such as just keeping a high traffic in their site thus instead of having loyal customers who spend a lot of money, they can manage big volume of customers who each spends a good sum of money.

# Questions

1. Is the problem statement that you defined at the beginning measurable? Give your opinion!
2. What is meant by customer/user retention?
3. If The Look platform makes additional features, will you recommend/prioritize loyal users? what are your considerations?

# Answers

1. The problem statement that has been defined at the beginning is measurable because from the problems that has been determined, it is clear what must be achieved in the next period and how much improvement is expected. In this case, the customer retention that is "clean" which must be increased by 2% at the end of the year.

2. Customer/user retention is defined as how many customers/users keeps coming back to use the services provided over a period of time.

3. The addition of a new feature should either be applied after they have managed to manage a stable rate of customer retention, or in efforts of increasing and maintaining the customer retention of the loyal users. Assuming that they're adding new features for said reasons, they could offer or prioritize the new features to loyal users in order to give them some kind of exclusivity and an attempt to maintain existing loyal customers.